## MODEL TESTING

## Libraries, Constants, Functions

In [ ]:
from libraries import *
from constants import *
from functions import *

gmaps = googlemaps.Client(key = GOOGLE_API_KEY)
CLOSEST_AEDS = 8

## Calculating Cost Matrices

In [ ]:
# Load aeds, cards and possible_locations and optimal_indicators

aeds = {}
cards = {}
possible_locations = {}
optimal_indicators = {}

for city in cities:
    os.chdir(data_path + clean_path)
    aeds[city] = pd.read_csv(city + "_aeds.csv")
    cards[city] = pd.read_csv(city + "_cards_test.csv")

    os.chdir(data_path + possible_locations_path)
    possible_locations[city] = pd.read_csv(city + "_possible_locations.csv")

    os.chdir(data_path + optimal_indicators_path)
    optimal_indicators[city] = pd.read_csv(city + "_optimal_indicators.csv")

In [ ]:
# Keep only coordinate columns

combined_locations = {}

for city in cities:
    aeds[city] = aeds[city][['latitude', 'longitude']]
    possible_locations[city][['latitude', 'longitude']] = possible_locations[city]['geometry'].apply(
        lambda x: pd.Series(get_coordinates_from_geometry(x))
    )
    possible_locations[city] = possible_locations[city][['latitude', 'longitude']]
    
    combined_locations[city] = pd.concat([aeds[city], possible_locations[city]], ignore_index=True)
    flag = optimal_indicators[city]["SelectionStatus"]
    optimal_locations = combined_locations[city][flag == True]
    combined_locations[city] = pd.concat([aeds[city], optimal_locations], ignore_index=True)
    
    cards[city] = cards[city][['latitude', 'longitude']]

In [ ]:
# Calculate cost matrix

cost_matrices = {}

for city in cities:
    print("Current city: " + city)
    
    # Euclidean distance matrix
    distance_matrix = cdist(cards[city], combined_locations[city], metric = 'euclidean')  # Transposed
    flag_matrix = np.zeros_like(distance_matrix)
    
    for row in range(distance_matrix.shape[0]):
        row_indices = np.argsort(distance_matrix[row, :])[:CLOSEST_AEDS]
        flag_matrix[row, row_indices] = 1
    
    # Column names
    cards_str = cards[city].apply(lambda x: f"{x['latitude']}, {x['longitude']}", axis = 1)
    flag_matrix = pd.DataFrame(flag_matrix, index = cards_str, columns = combined_locations[city].apply(tuple, axis = 1))
    flag_matrix.replace(0, INF_LENGTH, inplace=True)

    confirmation = input(f"This will initialize {len(flag_matrix) * CLOSEST_AEDS} API requests. Are you sure? (yes/no): ")
    if confirmation == "yes":            
        # Replace all marked cells with real calculated distances
        cost_matrices[city] = replace_1_with_distances(flag_matrix)
    else:
        print("OK. Will not procced.\n")

## Evaluation of coverage

In [ ]:
coverage_data = []

for city in cities:
    cost_matrix = cost_matrices[city]
    cost_matrix = cost_matrix.apply(pd.to_numeric, errors='coerce')

    # Split the cost matrix by old and new AED placements
    mid_index = cost_matrix.shape[1] // 2

    old_aeds_cost_matrix = cost_matrix.iloc[:, :mid_index]
    new_aeds_cost_matrix = cost_matrix.iloc[:, mid_index:]

    # Calculate if there's an AED within 150m for old and new placements
    old_aed_near_card = old_aeds_cost_matrix.apply(lambda row: (row < COVERAGE_RADIUS).any(), axis=1)
    new_aed_near_card = new_aeds_cost_matrix.apply(lambda row: (row < COVERAGE_RADIUS).any(), axis=1)
    
    coverage_data.append({
        'city': city,
        'old_coverage': old_aed_near_card.mean() * 100,
        'new_coverage': new_aed_near_card.mean() * 100
    })

os.chdir(data_path + app_path)
pd.DataFrame(coverage_data).to_csv("coverage.csv", index = False)

## Creating app_data

In [ ]:
os.chdir(data_path + clean_path)
all_dataframes = []

for city in cities:
    cost_matrix = cost_matrices[city]

    mid_index = cost_matrix.shape[1] // 2

    # Taking column names, not the cost matrices
    old_aed_columns = cost_matrix.columns[:mid_index]
    old_aeds_locations = pd.DataFrame(old_aed_columns, columns=["Old AEDs"])
    old_aeds_locations[['lat', 'lon']] = old_aeds_locations['Old AEDs'].apply(make_coordinates_tuple).apply(pd.Series)
    old_aeds_locations.drop(columns=['Old AEDs'], inplace=True)
    
    new_aed_columns = cost_matrix.columns[mid_index:]
    new_aeds_locations = pd.DataFrame(new_aed_columns, columns=["New AEDs"])
    new_aeds_locations[['lat', 'lon']] = new_aeds_locations['New AEDs'].apply(make_coordinates_tuple).apply(pd.Series)
    new_aeds_locations.drop(columns=['New AEDs'], inplace=True)

    # Importing cards
    cards = pd.read_csv(city + "_cards_test.csv")[['latitude', 'longitude']]
    cards.rename(columns={'latitude': 'lat'}, inplace=True)
    cards.rename(columns={'longitude': 'lon'}, inplace=True)

    new_aeds_cost_matrix = cost_matrix.iloc[:, mid_index:].apply(pd.to_numeric, errors='coerce')
    min_indices = np.argmin(new_aeds_cost_matrix.to_numpy(), axis=1)

    cards = pd.DataFrame({
        'lat': cards['lat'].values,
        'lon': cards['lon'].values,
        'aed_lat': new_aeds_locations.iloc[min_indices]['lat'].values,
        'aed_lon': new_aeds_locations.iloc[min_indices]['lon'].values
    })
    cards['distance'] = None
    for i, which_min_index in enumerate(min_indices):
        cards.at[i, 'distance'] = new_aeds_cost_matrix.iat[i, which_min_index]

    # Add type and city columns
    cards['type'] = 'card'
    old_aeds_locations['type'] = 'old_aed'
    new_aeds_locations['type'] = 'new_aed'
    cards['city'] = city
    old_aeds_locations['city'] = city
    new_aeds_locations['city'] = city

    # Add missing columns to old_aeds_locations and new_aeds_locations
    old_aeds_locations['aed_lat'] = np.nan
    old_aeds_locations['aed_lon'] = np.nan
    old_aeds_locations['distance'] = np.nan
    new_aeds_locations['aed_lat'] = np.nan
    new_aeds_locations['aed_lon'] = np.nan
    new_aeds_locations['distance'] = np.nan

    # Reorder columns to match the cards dataframe
    old_aeds_locations = old_aeds_locations[['lat', 'lon', 'aed_lat', 'aed_lon', 'distance', 'type', 'city']]
    new_aeds_locations = new_aeds_locations[['lat', 'lon', 'aed_lat', 'aed_lon', 'distance', 'type', 'city']]
    cards = cards[['lat', 'lon', 'aed_lat', 'aed_lon', 'distance', 'type', 'city']]

    # Concatenate dataframes for the current city
    all_dataframes.append(pd.concat([cards, old_aeds_locations, new_aeds_locations], ignore_index=True))

os.chdir(data_path + app_path)
app_data = pd.concat(all_dataframes, ignore_index = True)
app_data.to_csv("app_data.csv", index = False)